In [834]:
from plotly.subplots import make_subplots
import dash
from dash import html, dcc, dash_table, callback_context
import plotly.graph_objects as go
import dash_trich_components as dtc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import datetime
import sqlite3

In [835]:
from dash import jupyter_dash

# jupyter_dash.default_mode="inline"
jupyter_dash.default_mode = "external"

### Load main data into a pandas DataFrame

In [836]:
con = sqlite3.connect("stonks.db", check_same_thread=False)
cur = con.cursor()

In [837]:
main_query = """
SELECT t.symbol,
    t.name,
    s.name sector,
    DATE(d.date, 'unixepoch') date,
    p.open,
    p.high,
    p.low,
    p.close,
    p.volume,
    e.name exchange,
    tt.name type,
    c.iso_code currency
FROM price p
    JOIN ticker t ON p.ticker_id = t.id
    JOIN date d ON p.date_id = d.id
    JOIN sector s ON t.sector_id = s.id
    JOIN exchange e ON t.exchange_id = e.id
    JOIN currency c ON t.currency_id = c.id
    JOIN ticker_type tt ON t.ticker_type_id = tt.id
"""

In [838]:
# # Create DataFrame from SQL query
# main_table = pd.read_sql_query(main_query, con)
# # Make sure the date is in datetime format
# main_table.date = pd.to_datetime(main_table.date)
# main_table.head()

In [839]:
# Create DataFrame from SQL query
main_table = pd.read_sql_query(
    main_query,
    con,
    parse_dates=["date"],
    dtype={
        "symbol": "category",
        "name": "category",
        "sector": "category",
        "exchange": "category",
        "type": "category",
        "currency": "category",
    },
)
main_table.head()

,symbol,name,sector,date,open,high,low,close,volume,exchange,type,currency
0,AAPL,Apple Inc.,Technology,2022-07-01,135.232487,138.214679,134.854752,138.105331,71051600,NMS,EQUITY,USD
1,AAPL,Apple Inc.,Technology,2022-07-05,136.952245,140.769448,136.117219,140.719742,73353800,NMS,EQUITY,USD
2,AAPL,Apple Inc.,Technology,2022-07-06,140.510982,143.264529,140.242581,142.071655,74064300,NMS,EQUITY,USD
3,AAPL,Apple Inc.,Technology,2022-07-07,142.439444,145.680103,142.429509,145.481293,66253700,NMS,EQUITY,USD
4,AAPL,Apple Inc.,Technology,2022-07-08,144.397758,146.674173,144.139306,146.167191,64547800,NMS,EQUITY,USD


Experimenting: filtering in pandas with all data loaded vs filtering with multiple dynamic SQL queries. pandas wins as all data is loaded into memory once. This behavior may change if the space occupied by the database exceeds available ram

In [840]:
%%script echo W
%%timeit
ticker = main_table[main_table.symbol == 'AAPL'].set_index('date')


W


In [841]:
ticker_query = """
SELECT t.symbol,
    t.name,
    s.name sector,
    DATE(d.date, 'unixepoch') date,
    p.open,
    p.high,
    p.low,
    p.close,
    p.volume,
    e.name exchange,
    tt.name type,
    c.iso_code currency
FROM price p
    JOIN ticker t ON p.ticker_id = t.id
    JOIN date d ON p.date_id = d.id
    JOIN sector s ON t.sector_id = s.id
    JOIN exchange e ON t.exchange_id = e.id
    JOIN currency c ON t.currency_id = c.id
    JOIN ticker_type tt ON t.ticker_type_id = tt.id
WHERE t.symbol = 'AAPL'
"""

In [842]:
%%script echo L
%%timeit
ticker = pd.read_sql_query(ticker_query, con).set_index('date')


L


### Develope figures to be used in dashboard

In [843]:
# Define color variables
green = "#079A80"
red = "#F23645"
# Filter data for specific ticker
symbol = "AAPL"
ticker = main_table[main_table.symbol == symbol].set_index("date")
ticker.head()

,symbol,name,sector,open,high,low,close,volume,exchange,type,currency
date,,,,,,,,,,,
2022-07-01,AAPL,Apple Inc.,Technology,135.232487,138.214679,134.854752,138.105331,71051600,NMS,EQUITY,USD
2022-07-05,AAPL,Apple Inc.,Technology,136.952245,140.769448,136.117219,140.719742,73353800,NMS,EQUITY,USD
2022-07-06,AAPL,Apple Inc.,Technology,140.510982,143.264529,140.242581,142.071655,74064300,NMS,EQUITY,USD
2022-07-07,AAPL,Apple Inc.,Technology,142.439444,145.680103,142.429509,145.481293,66253700,NMS,EQUITY,USD
2022-07-08,AAPL,Apple Inc.,Technology,144.397758,146.674173,144.139306,146.167191,64547800,NMS,EQUITY,USD


Japanese candlestick chart with volume

In [844]:
# Add color column for plotting
ticker["color"] = green
ticker["color"] = ticker.color.where(ticker["close"] > ticker["open"], red).astype(
    "category"
)
ticker.head()

,symbol,name,sector,open,high,low,close,volume,exchange,type,currency,color
date,,,,,,,,,,,,
2022-07-01,AAPL,Apple Inc.,Technology,135.232487,138.214679,134.854752,138.105331,71051600,NMS,EQUITY,USD,#079A80
2022-07-05,AAPL,Apple Inc.,Technology,136.952245,140.769448,136.117219,140.719742,73353800,NMS,EQUITY,USD,#079A80
2022-07-06,AAPL,Apple Inc.,Technology,140.510982,143.264529,140.242581,142.071655,74064300,NMS,EQUITY,USD,#079A80
2022-07-07,AAPL,Apple Inc.,Technology,142.439444,145.680103,142.429509,145.481293,66253700,NMS,EQUITY,USD,#079A80
2022-07-08,AAPL,Apple Inc.,Technology,144.397758,146.674173,144.139306,146.167191,64547800,NMS,EQUITY,USD,#079A80


In [845]:
# Create figure with secondary y-axis (for the Volume chart)
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot candlesticks (price)
fig.add_trace(
    go.Candlestick(
        x=ticker.index,
        open=ticker.open,
        high=ticker.high,
        low=ticker.low,
        close=ticker.close,
        name="Price",
        increasing_line_color=green,
        decreasing_line_color=red,
    ),
    secondary_y=True,
)

# Plot bar chart (volume)
fig.add_trace(
    go.Bar(
        x=ticker.index,
        y=ticker.volume,
        name="Volume",
        marker_color=ticker.color,
        marker_opacity=0.5,
    ),
    secondary_y=False,
)

fig.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="grey",
    margin=dict(l=7, r=7, b=7, t=7),
    showlegend=False,
    hovermode="x unified",
    xaxis=dict(
        showgrid=False,
        range=["2023-01-01", "2023-07-01"],  # change default plot range
        rangeslider=dict(visible=True, bgcolor="black"),
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends
            # dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
        ],
        rangeselector=dict(
            bgcolor="black",
            buttons=list(
                [
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=3, label="3M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            ),
        ),
        # type="date",
    ),
    yaxis=dict(
        visible=False,
        showgrid=False,
        range=[0, ticker.volume.max() * 4],
        # scaleanchor="y2",
        # scaleratio=0.0000001,
        # constraintoward="bottom",
        rangemode="tozero",
    ),
    yaxis2=dict(
        showgrid=False,
        # tickprefix="$"
    ),
)

52-week (1 year) closing price line chart

In [846]:
ticker.close.tail(15)

date
2023-07-17    193.990005
2023-07-18    193.729996
2023-07-19    195.100006
2023-07-20    193.130005
2023-07-21    191.940002
2023-07-24    192.750000
2023-07-25    193.619995
2023-07-26    194.500000
2023-07-27    193.220001
2023-07-28    195.830002
2023-07-31    196.449997
2023-08-01    195.610001
2023-08-02    192.580002
2023-08-03    191.169998
2023-08-04    181.990005
Name: close, dtype: float64

In [847]:
# Resample to weekly close, with label from the beginning and value from the end of the week (with Monday as the start of the week)
ticker.resample(
    "W-MON",
    closed="left",
    label="left",
)["close"].last().tail()

date
2023-07-03    190.679993
2023-07-10    190.690002
2023-07-17    191.940002
2023-07-24    195.830002
2023-07-31    181.990005
Freq: W-MON, Name: close, dtype: float64

In [848]:
# The output from this resample operation feeds the weekly closing price chart
weekly_52 = (
    ticker.resample(
        "W-MON",
        closed="left",
        label="left",
    )["close"]
    .last()
    .iloc[-52:]
)

# Plot 52 week line chart (price)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=weekly_52.index,
        y=weekly_52.values,
        line_color="grey",
        line_width=3,
    )
)
fig.update_layout(
    title={"text": "Weekly Chart", "y": 0.9},
    font={"size": 9, "color": "grey"},
    plot_bgcolor="black",
    paper_bgcolor="black",
    margin=dict(l=7, r=7, b=7, t=7),
    showlegend=False,
)
fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_yaxes(showticklabels=False, showgrid=False)

Current price vs 52-week high/low indicator (speedometer)
###### Exchanges average about 252 trading days a year. This is from 365.25 (days on average per year) * 5/7 (proportion work days per week) - 6 (weekday holidays) - 4*5/7 (fixed date holidays) ≈ 252

In [849]:
# Speedometer chart which indicates the tickers current price compared to it's 52-week high/lows
df_52_week_low = ticker.close[-252:].min()
df_52_week_high = ticker.close[-252:].max()
current_price = ticker.iat[-1, ticker.columns.get_loc("close")]

# Plot gauge (speedometer)
fig = go.Figure()
fig.add_trace(
    go.Indicator(
        value=current_price,
        mode="gauge",
        gauge={
            "axis": {"range": [df_52_week_low, df_52_week_high]},
            "bar": {"color": "grey"},
        },
        domain={"x": [0, 1], "y": [0, 1]},
    )
)

# Plot current price
fig.add_trace(
    go.Indicator(
        value=current_price,
        mode="number",
        number={"valueformat": ".2f", "font_size": 77},
        domain={"x": [0.5, 0.5], "y": [0.5, 0.5]},
    )
)

# Plot colored percentage above the 52-week low
fig.add_trace(
    go.Indicator(
        value=current_price,
        mode="delta",
        delta={"reference": df_52_week_low, "relative": True, "valueformat": ".2%"},
        title={"text": "Above Low", "font_size": 27},
        domain={"x": [0.35, 0.45], "y": [0, 0.3]},
    )
)

# Plot colored percentage below the 52-week high
fig.add_trace(
    go.Indicator(
        value=current_price,
        mode="delta",
        delta={"reference": df_52_week_high, "relative": True, "valueformat": ".2%"},
        title={"text": "Below High", "font_size": 27},
        domain={"x": [0.55, 0.65], "y": [0, 0.3]},
    )
)

fig.update_layout(
    title={"text": "52-Week High/Low", "y": 0.9},
    font={"size": 9, "color": "grey"},
    plot_bgcolor="black",
    paper_bgcolor="black",
    margin=dict(l=7, r=7, b=7, t=21),
    showlegend=False,
)

### Initiate dash app

In [850]:
app = dash.Dash(__name__)
# theme = dbc.themes.CYBORG
# theme = dbc.themes.SUPERHERO
theme = dbc.themes.VAPOR
app = dash.Dash(external_stylesheets=[theme])
# app = dash.Dash(__name__)

Compare SQL queries that select the change of a ticker ordered by the absolute value (for the main carousel component)

In [851]:
%%script echo tho I think this version is more readable, the other is more performant
%%timeit
change_query = """
WITH growth AS (
    SELECT symbol,
        date,
        close,
        row_num
    FROM (
            SELECT t.symbol,
                d.date,
                p.close,
                ROW_NUMBER() OVER (
                    PARTITION BY t.symbol
                    ORDER BY d.date DESC
                ) AS row_num
            FROM price p
                JOIN ticker t ON p.ticker_id = t.id
                JOIN date d ON p.date_id = d.id
        )
    WHERE row_num <= 2
)
SELECT g1.symbol,
    100 * (g1.close - g2.close) / g2.close AS growth_percentage
FROM growth g1
    JOIN growth g2 ON g1.symbol = g2.symbol
    AND g1.row_num = g2.row_num - 1
ORDER BY ABS(growth_percentage) DESC
LIMIT 10;
"""
cur.execute(change_query).fetchmany(3)


tho I think this version is more readable, the other is more performant


In [852]:
change_query = """
WITH growth AS (
    SELECT t.symbol,
        d.date,
        p.close
    FROM price p
        JOIN ticker t ON p.ticker_id = t.id
        JOIN date d ON p.date_id = d.id
)
SELECT g1.symbol,
    100 * (g1.close - g2.close) / g2.close AS growth_percentage
FROM growth g1
    JOIN (
        SELECT symbol,
            MAX(date) today
        FROM growth
        GROUP BY symbol
    ) t ON g1.symbol = t.symbol
    AND g1.date = t.today
    JOIN growth g2 ON g1.symbol = g2.symbol
    AND g2.date = (
        SELECT MAX(date)
        FROM growth
        WHERE symbol = g1.symbol
            AND date < g1.date
    )
ORDER BY ABS(growth_percentage) DESC
LIMIT 10
"""
cur.execute(change_query).fetchmany(3)

[('AAPL', -4.802004898105658),
 ('NVDA', 3.6254930548998683),
 ('META', 1.935854875381768)]

Define sector and symbol queries (for dropdown components)

In [853]:
# list all sectors
sector_query = """
SELECT name
FROM sector
ORDER BY name
"""
cur.execute(sector_query).fetchmany(3)

[('Communication Services',), ('Consumer Cyclical',), ('Consumer Defensive',)]

In [854]:
# list ticker symbols in sector
symbols_query = """
SELECT symbol
FROM ticker t
    JOIN sector s ON s.id = t.sector_id
WHERE s.name = ?
ORDER BY symbol
"""
cur.execute(symbols_query, ("Technology",)).fetchmany(3)

[('AAPL',), ('ADBE',), ('AVGO',)]

### Define dash app layout components

Top change carousel

In [855]:
# Accepts a list of elements (list comp of html divs in this case) to "carouse" through
carousel = dtc.Carousel(
    [
        html.Div(
            [
                # Ticker symbol
                html.Span(symbol, style={"marginRight": "10px"}),
                # Change (colored)
                html.Span(
                    f"{'+' if change > 0 else ''}{change:.2f}%",
                    style={"color": "green" if change > 0 else "red"},
                ),
            ]
        )
        for symbol, change in cur.execute(change_query)
    ],
    id="main-carousel",
    autoplay=True,
    slides_to_show=5,
)

Dropdowns

In [856]:
# The economic sectors dropdown
dropdown_1 = dcc.Dropdown(
    options=[sector[0] for sector in cur.execute(sector_query)],
    value="Technology",
    id="sector-dropdown",
    style={"width": "330px", "backgroundColor": "rgba(0,0,0,0)"},
    # placeholder="Select Economic Sector",
    searchable=False,
    clearable=False,
)
# The symbols dropdown
dropdown_2 = dcc.Dropdown(
    # options will be filled by callback
    # Default ticker - first ticker of sector, chosen by callback
    id="symbol-dropdown",
    style={"width": "170px", "backgroundColor": "rgba(0,0,0,0)"},
    # placeholder="Select Ticker Symbol",
    searchable=False,
    clearable=False,
)

In [857]:
# Update symbols dropdown with tickers from the selected sector and select first value as default
# Calculate intra-sector data for "correlation-table"
@app.callback(
    Output("symbol-dropdown", "options"),
    Output("symbol-dropdown", "value"),
    Input("sector-dropdown", "value"),
)
def update_symbols_dropdown(sector):
    global intra_sector_corr, sector_table
    # Filter by sector and select necessary columns
    sector_table = main_table.loc[
        main_table.sector == sector, ["symbol", "date", "close"]
    ]
    # Convert to string from category
    sector_table["symbol"] = sector_table.symbol.astype(str)
    # Find the latest date that is shared by all symbols of the sector
    end_date = sector_table.groupby("symbol").date.max().min()
    # Subtract 90 days
    start_date = end_date - datetime.timedelta(90)
    # Filter the date
    sector_table = sector_table[
        (sector_table.date >= start_date) & (sector_table.date <= end_date)
    ]
    # Pivot and calculate correlations
    intra_sector_corr = sector_table.pivot(
        columns="symbol", index="date", values="close"
    ).corr()

    # Select ticker symbols from selected sector
    options = [symbol[0] for symbol in cur.execute(symbols_query, (sector,))]
    return options, options[0]

Update buttons

In [858]:
rounded_button = {
    # "border": None,
    "borderRadius": "21%",
    "padding": 0,
    "backgroundColor": "rgba(0,0,0,0)",
    # "color": "red",
    "textAlign": "center",
    "height": 30,
    "width": 70,
    # "margin": 20,
}

# This Div contains the update buttons for updating ticker data
buttons = html.Div(
    [
        html.Button(
            "Update",
            id="1-button",
            n_clicks=0,
            className="btn-outline-secondary",
            style=rounded_button,
        ),
        html.Button(
            "Updating",
            id="2-button",
            n_clicks=0,
            className="btn-outline-info",
            style=rounded_button,
        ),
        html.Button(
            "Success",
            id="3-button",
            n_clicks=0,
            className="btn-outline-success",
            style=rounded_button,
        ),
        html.Button(
            "Failure",
            id="4-button",
            n_clicks=0,
            className="btn-outline-danger",
            style=rounded_button,
        ),
    ],
    # style={"padding": "15px", "marginLeft": "35px"},
)

In [859]:
# Obtain the id of the clicked button
# # changed_id = [p['prop_id'] for p in callback_context.triggered][0]
# changed_id = callback_context.triggered_id

Main price chart (with indicators)

In [860]:
# The price (candlestick) chart
price_chart = dcc.Graph(id="price-chart")

In [861]:
# Checklist for selecting the indicators
indicators = dcc.Checklist(
    [
        "SMA",
        "EMA",
    ],
    inputStyle={"marginLeft": "15px", "marginRight": "5px"},
    id="indicators-checklist",
    style={"marginTop": "20px"},
    inline=True,
)

In [862]:
# Update price chart (with indicators)
@app.callback(
    Output("price-chart", "figure"),
    Input("symbol-dropdown", "value"),
    Input("indicators-checklist", "value"),
)
def update_price_chart(symbol, checklist_values):
    # Filter data by ticker symbol
    ticker = main_table[main_table.symbol == symbol].set_index("date")
    # Add color for plotting
    ticker["color"] = green
    ticker["color"] = ticker.color.where(ticker["close"] > ticker["open"], red).astype(
        "category"
    )
    # Each indicator will have its own color in the chart
    colors = {"SMA": "#1c90d4", "EMA": "#ad0026"}

    # Create figure with secondary y-axis (for the Volume chart)
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Plot candlesticks (price)
    fig.add_trace(
        go.Candlestick(
            x=ticker.index,
            open=ticker.open,
            high=ticker.high,
            low=ticker.low,
            close=ticker.close,
            name="Price",
            increasing_line_color=green,
            decreasing_line_color=red,
        ),
        secondary_y=True,
    )

    # Plot bar chart (volume)
    fig.add_trace(
        go.Bar(
            x=ticker.index,
            y=ticker.volume,
            name="Volume",
            marker_color=ticker.color,
            marker_opacity=0.5,
        ),
        secondary_y=False,
    )

    # Select latest date for specifying default range
    latest_date = ticker.index[-1]
    fig.update_layout(
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font_color="grey",
        margin=dict(l=7, r=7, b=7, t=7),
        showlegend=False,
        hovermode="x unified",
        transition_duration=1000,
        xaxis=dict(
            showgrid=False,
            range=[
                latest_date - datetime.timedelta(180),
                # ticker.index[-1] - relativedelta(months=6),
                latest_date + datetime.timedelta(1),
            ],  # default plot range - 180 days before latest update
            rangeslider=dict(visible=True, bgcolor="black"),
            rangebreaks=[
                dict(bounds=["sat", "mon"]),  # hide weekends
                # dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
            ],
            rangeselector=dict(
                bgcolor="rgba(0,0,0,0)",
                buttons=list(
                    [
                        dict(count=1, label="1M", step="month", stepmode="backward"),
                        dict(count=3, label="3M", step="month", stepmode="backward"),
                        dict(count=6, label="6M", step="month", stepmode="backward"),
                        dict(count=1, label="YTD", step="year", stepmode="todate"),
                        dict(count=1, label="1Y", step="year", stepmode="backward"),
                        dict(step="all"),
                    ]
                ),
            ),
            # type="date",
        ),
        yaxis=dict(
            visible=False,
            showgrid=False,
            range=[0, ticker.volume.max() * 4],
            # scaleanchor="y2",
            # scaleratio=0.0000001,
            # constraintoward="bottom",
            rangemode="tozero",
        ),
        yaxis2=dict(
            showgrid=False,
            # tickprefix="$"
        ),
    )

    # If the user has selected any of the indicators in the checklist, plot it on the chart
    if checklist_values != None:
        for indicator in checklist_values:
            if indicator in ticker.columns:
                pass
            # Calculate indicator values
            elif indicator == "SMA":
                ticker["SMA"] = ticker["close"].rolling(window=9).mean()
            else:
                ticker["EMA"] = (
                    ticker["close"].ewm(span=9, min_periods=9, adjust=False).mean()
                )
            # Plot indicator
            fig.add_trace(
                go.Scatter(
                    x=ticker.index,
                    y=ticker[indicator],
                    mode="lines",
                    name=indicator,
                    line={"color": colors[indicator], "width": 3},
                    opacity=0.7,
                ),
                secondary_y=True,
            )

    return fig

Price card with main ticker info

```
AMZN  
Amazon.com, Inc.	
                141.71 (USD)
               +2.13 (1.43%)
Exchange: NasdaqGS
Sector: Consumer Cyclical
```

<style>
red { all: unset; color: Red }
green { all: unset; color: Green }
blue { all: unset; color: Blue }
</style>
This is some <span style='color:red'>red</span> text  
This is some <green>green</green> text  
This is some <blue>blue</blue> text  

In [863]:
price_card = html.Div(
    [
        # Symbol
        html.H4(
            id="ticker-symbol",
            style={
                "textAlign": "left",
                "marginTop": 10,
                "marginBottom": -7,
            },
        ),
        # Name
        html.P(
            id="ticker-name",
            style={
                "fontSize": "12px",
                "textAlign": "left",
                "marginBottom": -7,
            },
        ),
        # Price and currency
        html.P(
            id="ticker-price",
            style={
                "fontSize": "27px",
                "textAlign": "right",
                "marginBottom": -7,
            },
        ),
        # Price change (style is specified in callback)
        html.P(
            id="ticker-change",
        ),
        # Exchange
        html.P(
            id="exchange-name",
            style={
                "textAlign": "left",
                "fontSize": "14px",
                "marginBottom": -3,
            },
        ),
        # Economic sector
        html.P(
            id="economic-sector",
            style={
                "textAlign": "left",
                "fontSize": "14px",
            },
        ),
    ],
    id="ticker-data",
    style={"height": "110px"},
)

In [864]:
# The following function will edit the values being displayed in the "ticker-data" Div
@app.callback(
    Output("ticker-symbol", "children"),
    Output("ticker-name", "children"),
    Output("ticker-price", "children"),
    Output("ticker-change", "children"),
    Output("ticker-change", "style"),
    Output("exchange-name", "children"),
    Output("economic-sector", "children"),
    Input("symbol-dropdown", "value"),
)
def update_symbol_data(symbol):
    ticker = main_table.loc[
        main_table.symbol == symbol, ["name", "close", "exchange", "sector", "currency"]
    ].tail(2)
    # Getting the chosen symbol's current price and its change in comparison to its previous value
    current_price = ticker.iat[-1, 1]
    change = (current_price / ticker.iat[-2, 1]) - 1
    return (
        symbol,
        ticker.iat[0, 0], # ticker name
        f"{current_price:.2f} ({ticker.iat[0, 4]})", # (currency)
        f"{'+' if change > 0 else ''}{change:.2%}",
        {
            "fontSize": "14px",
            "textAlign": "right",
            "marginBottom": -3,
            "color": "green" if change > 0 else "red",
        }, # set style color depending on price change
        f"Exchange: {ticker.iat[0, 2]}",
        f"Sector: {ticker.iat[0, 3]}",
    )

Intra-sector 90-day correlation coefficient table

In [865]:
# This DataTable contains intra-sector ticker prices and 90-day correlations
table_info = html.Div(
    [
        html.Label(
            "Intra-sector Data",
            style={"textAlign": "center"},
        ),
        dash_table.DataTable(
            id="correlation-table",
            style_cell={
                "font_size": "12px",
                "textAlign": "right",
                "padding-right": "7px",
            },
            style_cell_conditional=[
                {
                    "if": {"column_id": "symbol"},
                    "textAlign": "left",
                    "padding-left": "7px",
                },
                {"if": {"column_id": ["price", "90-day corr"]}, "width": "30%"},
            ],
            style_header={"backgroundColor": "rgba(0,0,0,0)"},
            style_data={"backgroundColor": "rgba(0,0,0,0)"},
            style_table={"height": "100px", "overflowY": "auto"},
            style_as_list_view=True,
        ),
    ]
)

In [866]:
sector_table = main_table[
    main_table.sector
    == main_table[main_table.symbol == "AAPL"].iat[
        0, main_table.columns.get_loc("sector")
    ]
].drop(
    columns=[
        "name",
        "sector",
        "open",
        "high",
        "low",
        "volume",
        "exchange",
        "type",
        "currency",
    ]
)
sector_table["symbol"] = sector_table.symbol.astype(str)
# sector_table

In [867]:
sector_table[sector_table.symbol != "AAPL"].symbol.unique()

array(['ADBE', 'AVGO', 'CSCO', 'MSFT', 'NVDA', 'ORCL'], dtype=object)

In [868]:
# Find the latest date that is shared by all symbols of the sector
end_date = sector_table.groupby("symbol").date.max().min()
# Subtract 90 days
start_date = end_date - datetime.timedelta(90)

In [869]:
# Filter the date
sector_table = sector_table[
    (sector_table.date >= start_date) & (sector_table.date <= end_date)
]

# Pivot for easy correlation calculations
pivoted_sector = sector_table.pivot(columns="symbol", index="date", values="close")
pivoted_sector

symbol,AAPL,ADBE,AVGO,CSCO,MSFT,NVDA,ORCL
date,,,,,,,
2023-04-03,165.940475,380.079987,639.065308,51.529251,286.603485,279.621063,93.201256
2023-04-04,165.401230,385.149994,631.048035,51.430000,286.553558,274.501587,93.280655
2023-04-05,163.533798,382.019989,625.676697,51.430000,283.719757,268.782196,94.163841
2023-04-06,164.432556,380.600006,619.340454,50.884140,290.963928,270.342010,95.185959
2023-04-10,161.806183,376.250000,624.065247,50.943691,288.758759,275.761475,93.432114
...,...,...,...,...,...,...,...
2023-06-26,185.270004,479.510010,821.630005,49.832115,328.600006,406.320007,116.371605
2023-06-27,188.059998,489.269989,848.400024,50.477226,334.570007,418.760010,117.427895
2023-06-28,189.250000,482.429993,847.940002,50.487148,335.850006,411.170013,116.122482


In [870]:
corr = pivoted_sector.corr()

In [871]:
corr.AAPL.drop("AAPL").to_frame().reset_index().rename(
    columns={"AAPL": "90-day corr"}
).sort_values(by="90-day corr", key=abs, ascending=False).round(3).to_dict("records")

[{'symbol': 'MSFT', '90-day corr': 0.932},
 {'symbol': 'NVDA', '90-day corr': 0.927},
 {'symbol': 'ORCL', '90-day corr': 0.924},
 {'symbol': 'AVGO', '90-day corr': 0.917},
 {'symbol': 'ADBE', '90-day corr': 0.836},
 {'symbol': 'CSCO', '90-day corr': 0.41}]

In [872]:
sector_corr = corr.AAPL.drop("AAPL").to_frame().round(3)
sector_corr

,AAPL
symbol,
ADBE,0.836
AVGO,0.917
CSCO,0.410
MSFT,0.932
NVDA,0.927
ORCL,0.924


In [873]:
sector_price = (
    sector_table.drop(columns="date").groupby("symbol").last().drop("AAPL").round(2)
)
sector_price

,close
symbol,
ADBE,488.99
AVGO,867.43
CSCO,51.35
MSFT,340.54
NVDA,423.02
ORCL,118.67


In [874]:
table = (
    sector_price.join(sector_corr)
    .reset_index()
    .rename(columns={"AAPL": "90-day corr", "close": "price"})
    .sort_values(by="90-day corr", key=abs, ascending=False)
)

In [875]:
table.to_dict("records")

[{'symbol': 'MSFT', 'price': 340.54, '90-day corr': 0.932},
 {'symbol': 'NVDA', 'price': 423.02, '90-day corr': 0.927},
 {'symbol': 'ORCL', 'price': 118.67, '90-day corr': 0.924},
 {'symbol': 'AVGO', 'price': 867.43, '90-day corr': 0.917},
 {'symbol': 'ADBE', 'price': 488.99, '90-day corr': 0.836},
 {'symbol': 'CSCO', 'price': 51.35, '90-day corr': 0.41}]

In [876]:
# Update the table
@app.callback(
    Output("correlation-table", "data"),
    Input("symbol-dropdown", "value"),
)
def update_intra_sector_table(symbol):
    # Filter intra-sector correlation data
    sector_corr = intra_sector_corr[symbol].drop(symbol).to_frame().round(3)
    # Filter intra-sector price data
    sector_price = (
        sector_table.drop(columns="date").groupby("symbol").last().drop(symbol).round(2)
    )
    # Combine into a single table
    table = (
        sector_price.join(sector_corr)
        .reset_index()
        .rename(columns={symbol: "90-day corr", "close": "price"})
        .sort_values(by="90-day corr", key=abs, ascending=False)
    )
    return table.to_dict("records")

52-week data

In [877]:
# Carousel showing 52-week data
carousel_52_week = dtc.Carousel(
    [
        dcc.Graph(id="52-week-price-chart"),
        dcc.Graph(id="52-week-high-low-indicator"),
    ],
    # responsive=True,
    slides_to_show=1,
    autoplay=True,
    speed=2000,
    style={
        "height": 230,
        "width": 270
    }
    
)

In [878]:
# This function is responsible for updating the weekly price chart and the gauge (speedometer) chart
@app.callback(
    Output("52-week-price-chart", "figure"),
    Output("52-week-high-low-indicator", "figure"),
    Input("symbol-dropdown", "value"),
)
def update_52_week_charts(symbol):
    # Filter data by ticker symbol
    ticker = main_table[main_table.symbol == symbol].set_index("date")

    # The output from this resample operation feeds the weekly closing price chart
    weekly_52 = (
        ticker.resample(
            "W-MON",
            closed="left",
            label="left",
        )["close"]
        .last()
        .iloc[-52:]
    )

    # Plot 52 week chart (price)
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=weekly_52.index,
            y=weekly_52.values,
            line_color="grey",
            line_width=3,
        )
    )
    fig.update_layout(
        title={"text": "Weekly Chart", "y": 0.9},
        font={"size": 9, "color": "grey"},
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        height=230,
        width=270,
        margin=dict(l=1, r=1, b=0, t=0),
        showlegend=False,
    )
    fig.update_xaxes(showticklabels=False, showgrid=False)
    fig.update_yaxes(showticklabels=False, showgrid=False)

    # Get 52-week low/high and current price
    df_52_week_low = ticker.close[-252:].min()
    df_52_week_high = ticker.close[-252:].max()
    current_price = ticker.iat[-1, ticker.columns.get_loc("close")]

    # Plot gauge (speedometer) chart which indicates the tickers current price compared to it's 52-week high/low
    fig2 = go.Figure()
    fig2.add_trace(
        go.Indicator(
            value=current_price,
            mode="gauge",
            gauge={
                "axis": {"range": [df_52_week_low, df_52_week_high]},
                "bar": {"color": "grey"},
            },
            domain={"x": [0, 1], "y": [0, 0.9]},
        )
    )

    # Plot current price
    fig2.add_trace(
        go.Indicator(
            value=current_price,
            mode="number",
            number={"valueformat": ".2f", "font_size": 27},
            domain={"x": [0.5, 0.5], "y": [0.35, 0.5]},
        )
    )

    # Plot colored percentage above the 52-week low
    fig2.add_trace(
        go.Indicator(
            value=current_price,
            mode="delta",
            delta={"reference": df_52_week_low, "relative": True, "valueformat": ".2%", "font_size": 13},
            title={"text": "Above Low", "font_size": 13},
            domain={"x": [0.27, 0.37], "y": [0, 0.35]},
        )
    )

    # Plot colored percentage below the 52-week high
    fig2.add_trace(
        go.Indicator(
            value=current_price,
            mode="delta",
            delta={
                "reference": df_52_week_high,
                "relative": True,
                "valueformat": ".2%",
                "font_size": 13
            },
            title={"text": "Below High", "font_size": 13},
            domain={"x": [0.63, 0.73], "y": [0, 0.35]},
        )
    )

    fig2.update_layout(
        title={"text": "52-Week High/Low Indicator", "y": 0.9},
        font={"size": 9, "color": "grey"},
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        height=230,
        width=270,
        margin=dict(l=1, r=1, b=0, t=0),
        showlegend=False,
    )

    return fig, fig2

Storage

In [879]:
%%script echo later

# Store intermediate values
storage = dcc.Store(id="storage")

@app.callback(Output("storage", "data"), Input("symbol-dropdown", "value"))
def filter_data(symbol):
    return


later


### Layout and callbacks

In [880]:
app.layout = dbc.Container(
    [
        # A carousel for 10 tickers with the largest absolute change is created
        dbc.Row([dbc.Col([carousel], width=12)]),
        dbc.Row(
            [
                # The column below will occupy 75% of the width available in the dashboard.
                dbc.Col(
                    [
                        # This row holds the dropdowns responsible for sector and ticker selection
                        dbc.Row(
                            [
                                # Column 1
                                dbc.Col(
                                    [dropdown_1],
                                    width=6,
                                ),
                                # Column 2
                                dbc.Col(
                                    [dropdown_2],
                                    width=6,
                                ),
                            ]
                        ),
                        # All the three components are placed inside this dbc.Row.
                        dbc.Row(
                            [
                                dbc.Col(
                                    [
                                        dcc.Loading(
                                            [price_chart],
                                            id="loading-price-chart",
                                            type="circle",
                                            color="#1F51FF",
                                        )
                                    ],
                                    width=12,
                                ),
                            ]
                        ),
                        # Next, this row will store the time span buttons as well
                        # as the indicators checklist
                        dbc.Row(
                            [
                                dbc.Col([buttons], width=4),
                                dbc.Col([indicators], width=8),
                            ]
                        ),
                    ],
                    width=9,
                ),
                # This other column occupies 25% of the dashboard's width.
                dbc.Col(
                    [
                        dbc.Row([dbc.Col([price_card], width=12)]),
                        dbc.Row(
                            [
                                dbc.Col(
                                    [
                                        dcc.Loading(
                                            table_info,
                                            id="loading-table",
                                            type="dot",
                                            color="#1F51FF",
                                        )
                                    ],
                                    width=12,
                                )
                            ],
                            style={"marginTop": "28px"},
                        ),
                        dbc.Row([dbc.Col([carousel_52_week], width=12)]),
                    ],
                    width=3,
                ),
            ],
        ),
        # storage,
    ],
    fluid=True,
    style={"height": "100%", "width": "100%", "margin": 0, "overflow": "hidden"},
)

In [881]:
# List layout ids
elements = []
for pg in dash.page_registry:
    data = dash.page_registry[pg]["layout"]
    try:
        for i in data:
            elements.append(i)
    except:
        pass
data = dash.get_app().layout
try:
    for i in data:
        elements.append(i)
except:
    pass
print(elements)

['main-carousel', 'sector-dropdown', 'symbol-dropdown', 'loading-price-chart', 'price-chart', '1-button', '2-button', '3-button', '4-button', 'indicators-checklist', 'ticker-data', 'ticker-symbol', 'ticker-name', 'ticker-price', 'ticker-change', 'exchange-name', 'economic-sector', 'loading-table', 'correlation-table', '52-week-price-chart', '52-week-high-low-indicator']


In [882]:
if __name__ == "__main__":
    app.run(debug=True)

Dash app running on http://127.0.0.1:8050/


fix:
* fix 52-week charts formatting
* html label textAlign not working dash
* move buttons to top right on dropdown row
* dropdown coloring when expanded
* indicator period selection

* use dbc.DropdownMenu instead of dcc.Dropdown
* use dbc.Button
* use dbc.Checklist with switch=True, inline=True instead of dcc.Checklist
* add dbc.Input for selecting indicator period
* keep and rename ema, add period, remove sma
* add atr with period on separate graph bellow
* mitigate potential issues of using a global variable in a multi-user environment with dcc.Store
* try replacing all unneeded SQL queries with pandas (timeit)?

see All-in-One Components from https://community.plotly.com/t/dash-2-0-prerelease-candidate-available/55861  
see https://dash.plotly.com/urls  
see https://dash.plotly.com/deployment